In [17]:
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel,Field
from typing import Literal

load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id='meta-llama/Llama-3.3-70B-Instruct',
    task='text-generation'
)

model = ChatHuggingFace(llm=llm)
parser = StrOutputParser()

In [18]:
class Feedback(BaseModel):
    sentiment: Literal['positive','negative'] = Field("Give the Sentiment of the feedback")
    
parser2 = PydanticOutputParser(pydantic_object=Feedback)

In [19]:
prompt1=PromptTemplate(
    template='Classify the sentiment of the following feedback text into postive or negative \n {feedback} \n {format_ins}',
    input_variables=['feedback'],
    partial_variables={'format_ins':parser2.get_format_instructions()}
)

classifier_chain = prompt1 | model | parser2

In [20]:
prompt2=PromptTemplate(
    template="Write an appropiate response to this positive Feedback \n {feedback}",
    input_variables=["feedback"]
)

prompt3=PromptTemplate(
    template="Write an appropiate response to this negative Feedback \n {feedback}",
    input_variables=["feedback"]
)

In [21]:
branch_chain = RunnableBranch(
    (lambda x:x.sentiment=='positive', prompt2 | model | parser),
    (lambda x:x.sentiment=='negative', prompt3 | model | parser),
    RunnableLambda(lambda x: "could not found the sentiment")
)

In [22]:
chain = classifier_chain | branch_chain

In [24]:
print(chain.invoke({'feedback': 'This is the worst smart phone'}))

Here's a potential response to negative feedback:

"Thank you for taking the time to share your concerns with us. We apologize that our [product/service] did not meet your expectations. We take all feedback seriously and would like to make things right. Can you please provide more details about your experience so we can better understand what went wrong? We'll do our best to address the issue and prevent it from happening again in the future. Your satisfaction is our top priority, and we appreciate your feedback in helping us improve."

This response:

* Acknowledges the customer's negative experience
* Apologizes for the issue
* Shows appreciation for the feedback
* Requests more information to understand the issue
* Expresses a commitment to making things right and improving the product/service

Feel free to modify it to fit your specific needs and tone!


In [25]:
chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
  +-----------------+    
  | ChatHuggingFace |    
  +-----------------+    
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |        
       +--------+        
            *            
            *            
            *            
    +--------------+     
    | BranchOutput |     
    +--------------+     
